# 1.Graph classification task
# Dataset: Molecular_Sample.csv

### 1.1. Extract graph features with eigenvector centrality, Katz centrality, Laplacian centrality
### 1.2.Train model (SVM/MLP) with graph features as concatenation of above graph features,  with test_size = 0.3

In [2]:
import pandas as pd
import numpy as np
from sklearn import svm
from rdkit import Chem
from sklearn.model_selection import train_test_split
import networkx as nx
from sklearn.preprocessing import StandardScaler

# Load the dataset
data = pd.read_csv("./data/Molecular_Sample.csv")

# Preprocess the data
graphs = []
for smile in data['smiles']:
    mol = Chem.MolFromSmiles(smile)
    if mol is not None:
        graph = Chem.rdmolops.GetAdjacencyMatrix(mol)
        graphs.append(graph)

# Extract node features (eigenvector centrality, Katz centrality, Laplacian centrality)
node_features = []
for graph in graphs:
    G = nx.from_numpy_array(graph)
    eigenvector_centrality = list(nx.eigenvector_centrality_numpy(G).values())
    katz_centrality = list(nx.katz_centrality_numpy(G).values())
    laplacian_centrality = list(nx.laplacian_centrality(G).values())
    features = np.column_stack((eigenvector_centrality, katz_centrality, laplacian_centrality))
    node_features.append(features)

# Aggregate node features (sum)
aggregated_features = [np.sum(features, axis=0) for features in node_features]

# Reshape the input arrays
X_train, X_test, y_train, y_test = train_test_split(aggregated_features, data['p_np'], test_size=0.3, random_state=42)
X_train_reshaped = np.array(X_train).reshape(-1, 3)  # 3 centrality features
X_test_reshaped = np.array(X_test).reshape(-1, 3)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_reshaped)
X_test_scaled = scaler.transform(X_test_reshaped)

# Train SVM classifier
clf = svm.SVC(kernel='sigmoid')
clf.fit(X_train_scaled, y_train)

# Predict on test set
y_pred = clf.predict(X_test_scaled)

# Evaluate accuracy
accuracy = clf.score(X_test_scaled, y_test)
print("Accuracy:", accuracy)


Accuracy: 0.25


# 2. Graph classification task
# Dataset: Molecular_Sample.csv

### Train model (SVM/MLP) with graph features as graphlet kernel from practice code W7 with test_size = 0.3

### Graphlet Kernel

In [43]:
import networkx as nx
import numpy as np
from itertools import combinations

def compute_graphlet_counts(G, k):
    """
    Compute the counts of k-node graphlets in the given graph.

    Parameters:
        G (networkx.Graph): The input graph.
        k (int): The size of the graphlets to count.

    Returns:
        dict: A dictionary containing the counts of k-node graphlets.
    """
    graphlet_counts = {graphlet: 0 for graphlet in range(1, k+1)}

    for node in G.nodes():
    # Generate the induced subgraph with node and its neighbors
        subgraph_nodes = [node] + list(G.neighbors(node))
        subgraph = G.subgraph(subgraph_nodes)

        # Count the occurrences of k-node graphlets in the subgraph
        for graphlet_size in range(1, k+1):
            for subgraph_nodes_subset in combinations(subgraph_nodes, graphlet_size):
                if G.subgraph(subgraph_nodes_subset).number_of_nodes() == graphlet_size:
                    graphlet_counts[graphlet_size] += 1

    return graphlet_counts

def compute_graphlet_kernel(G1, G2, k):
    """
    Compute the graphlet kernel between two graphs.

    Parameters:
        G1 (networkx.Graph): The first input graph.
        G2 (networkx.Graph): The second input graph.
        k (int): The size of the graphlets to count.

    Returns:
        float: The similarity score between the two graphs.
    """
    # Compute the counts of k-node graphlets in each graph
    graphlet_counts_G1 = compute_graphlet_counts(G1, k)
    graphlet_counts_G2 = compute_graphlet_counts(G2, k)

    # Compute the kernel value by comparing the counts of graphlets
    kernel_value = 0
    for graphlet in range(1, k+1):
        kernel_value += min(graphlet_counts_G1[graphlet], graphlet_counts_G2[graphlet])

    return kernel_value

In [48]:
import networkx as nx
import numpy as np
from itertools import combinations
from rdkit import Chem
from rdkit.Chem import AllChem

def mol_to_nx_graph(mol):
    """
    Convert RDKit molecule to NetworkX graph.

    Parameters:
        mol (RDKit molecule): The input molecule.

    Returns:
        networkx.Graph: The NetworkX graph representation of the molecule.
    """
    G = nx.Graph()

    # Add nodes for atoms
    for atom in mol.GetAtoms():
        G.add_node(atom.GetIdx(), atomic_num=atom.GetAtomicNum())

    # Add edges for bonds
    for bond in mol.GetBonds():
        G.add_edge(bond.GetBeginAtomIdx(), bond.GetEndAtomIdx(), bond_type=bond.GetBondType())

    return G

# Load molecular dataset
molecules = [
    "[Cl].CC(C)NCC(O)COc1cccc2ccccc12",
    "C(=O)(OC(C)(C)C)CCCc1ccc(cc1)N(CCCl)CCCl",
    "c1cccn2c1nc(c2)CCN",
    "Cc1onc(c2ccccc2Cl)c1C(=O)N[C@H]3[C@H]4SC(C)(C)[C@@H](N4C3=O)C(O)=O",
    "CCN1CCN(C(=O)N[C@@H](C(=O)N[C@H]2[C@H]3SCC(=C(N3C2=O)C(O)=O)CSc4nnnn4C)c5ccc(O)cc5)C(=O)C1=O",
    "CN(C)[C@H]1[C@@H]2C[C@H]3C(=C(O)c4c(O)cccc4[C@@]3(C)O)C(=O)[C@]2(O)C(=O)\C(=C(/O)NCN5CCCC5)C1=O",
    "Cn1c2CCC(Cn3ccnc3C)C(=O)c2c4ccccc14",
    "COc1ccc(cc1)[C@@H]2Sc3ccccc3N(CCN(C)C)C(=O)[C@@H]2OC(C)=O",
    "NC(N)=NC(=O)c1nc(Cl)c(N)nc1N",
    "OCC(C)(O)c1onc(c2ncn3c2CN(C)C(c4c3cccc4Cl)=O)n1",
    "CC1=CN([C@H]2C[C@H](F)[C@@H](CO)O2)C(=O)NC1=O",
    "CCC(=O)C(CC(C)N(C)C)(c1ccccc1)c2ccccc2",
    "CCN1N=NN(CCN2CCC(CC2)(COC)N(C(=O)CC)c3ccccc3)C1=O",
    "CN(C)C(=O)C(CCN1CCC(O)(CC1)c1ccc(Cl)cc1)(c1ccccc1)c1ccccc1",
    "CN1C2CCC1CC(C2)OC(=O)[C@H](CO)c3ccccc3",
    "COc1ccc(Cl)cc1C(=O)NCCc2ccc(cc2)[S](=O)(=O)NC(=O)NC3CCCCC3",
    "Nc1nnc(c(N)n1)c2cccc(Cl)c2Cl",
    "OC(C)(C)c1onc(c2ncn3c2CN(C)C(c4c3cccc4Cl)=O)n1",
    "CC(=O)Oc1ccccc1C(O)=O",
    "O=C1N=CN=C2NNC=C12",
    "CCCCC[C@H](O)/C=C/[C@H]1[C@H](O)CC(=O)[C@@H]1CCCCCCC(O)=O",
    "CN1C(=O)N(C)c2nc[nH]c2C1=O.CN3C(=O)N(C)c4nc[nH]c4C3=O.NCCN",
    "CCCCc1oc2ccccc2c1C(=O)c3cc(I)c(OCCN(CC)CC)c(I)c3",
    "O.O.O.CC1(C)S[C@@H]2[C@H](NC(=O)[C@H](N)c3ccc(O)cc3)C(=O)N2[C@H]1C(O)=O",
    "CC1(C)S[C@@H]2[C@H](NC(=O)[C@H](N)c3ccccc3)C(=O)N2[C@H]1C(O)=O",
    "C[C@]1(O)CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(C)[C@H]3CC[C@]12C"
]

# Convert molecules to NetworkX graphs
molecule_graphs = [mol_to_nx_graph(Chem.MolFromSmiles(mol)) for mol in molecules]

# Compute the graph kernel for all pairs of molecules
num_molecules = len(molecule_graphs)
graph_kernel_matrix = np.zeros((num_molecules, num_molecules))

for i in range(num_molecules):
    for j in range(num_molecules):
        kernel_value = compute_graphlet_kernel(molecule_graphs[i], molecule_graphs[j], k)
        graph_kernel_matrix[i, j] = kernel_value

# print("Graph Kernel Matrix:")
# print(graph_kernel_matrix)


In [34]:
import pandas as pd

data = pd.read_csv("./data/Molecular_Sample.csv")


In [46]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Extracting the 'p_np' column as class labels
class_labels = data['p_np'].tolist()

# Split data into features (graph kernel matrix) and labels
X = graph_kernel_matrix
y = class_labels

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Train an SVM classifier
clf = SVC(kernel='sigmoid')
clf.fit(X_train, y_train)

# Predict on test data
y_pred = clf.predict(X_test)

# Evaluate classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.625


### Random walk kernel

In [50]:
import networkx as nx
import random

def random_walk_kernel(graph1, graph2, walk_length=2, num_walks=20):
    # Initialize the kernel to zero
    kernel = 0

    # Perform random walks on both graphs
    for _ in range(num_walks):
        # Perform a random walk on the first graph
        walk1 = random_walk(graph1, walk_length)

        # Perform a random walk on the second graph
        walk2 = random_walk(graph2, walk_length)

        # Update the kernel if the walks have common subsequences
        if walk1 == walk2:
            kernel += 1

    # Normalize the kernel
    kernel /= num_walks

    return kernel

def random_walk(graph, walk_length):
    # Start the walk at a random node
    node = random.choice(list(graph.nodes()))

    # Perform the random walk
    walk = [node]
    for _ in range(walk_length - 1):
        neighbors = list(graph.neighbors(node))
        if neighbors:
            node = random.choice(neighbors)
            walk.append(node)
        else:
            break

    return tuple(walk)

In [53]:
import random
import networkx as nx
from itertools import combinations

# Define a function to convert a SMILES string to a NetworkX graph
def smiles_to_nx_graph(smiles):
    mol = Chem.MolFromSmiles(smiles)
    G = nx.Graph()
    for atom in mol.GetAtoms():
        G.add_node(atom.GetIdx(), atomic_num=atom.GetAtomicNum())
    for bond in mol.GetBonds():
        G.add_edge(bond.GetBeginAtomIdx(), bond.GetEndAtomIdx())
    return G

# Define the molecular dataset
molecules = [
    "[Cl].CC(C)NCC(O)COc1cccc2ccccc12",
    "C(=O)(OC(C)(C)C)CCCc1ccc(cc1)N(CCCl)CCCl",
    "c1cccn2c1nc(c2)CCN",
    "Cc1onc(c2ccccc2Cl)c1C(=O)N[C@H]3[C@H]4SC(C)(C)[C@@H](N4C3=O)C(O)=O",
    "CCN1CCN(C(=O)N[C@@H](C(=O)N[C@H]2[C@H]3SCC(=C(N3C2=O)C(O)=O)CSc4nnnn4C)c5ccc(O)cc5)C(=O)C1=O",
    "CN(C)[C@H]1[C@@H]2C[C@H]3C(=C(O)c4c(O)cccc4[C@@]3(C)O)C(=O)[C@]2(O)C(=O)\C(=C(/O)NCN5CCCC5)C1=O",
    "Cn1c2CCC(Cn3ccnc3C)C(=O)c2c4ccccc14",
    "COc1ccc(cc1)[C@@H]2Sc3ccccc3N(CCN(C)C)C(=O)[C@@H]2OC(C)=O",
    "NC(N)=NC(=O)c1nc(Cl)c(N)nc1N",
    "OCC(C)(O)c1onc(c2ncn3c2CN(C)C(c4c3cccc4Cl)=O)n1",
    "CC1=CN([C@H]2C[C@H](F)[C@@H](CO)O2)C(=O)NC1=O",
    "CCC(=O)C(CC(C)N(C)C)(c1ccccc1)c2ccccc2",
    "CCN1N=NN(CCN2CCC(CC2)(COC)N(C(=O)CC)c3ccccc3)C1=O",
    "CN(C)C(=O)C(CCN1CCC(O)(CC1)c1ccc(Cl)cc1)(c1ccccc1)c1ccccc1",
    "CN1C2CCC1CC(C2)OC(=O)[C@H](CO)c3ccccc3",
    "COc1ccc(Cl)cc1C(=O)NCCc2ccc(cc2)[S](=O)(=O)NC(=O)NC3CCCCC3",
    "Nc1nnc(c(N)n1)c2cccc(Cl)c2Cl",
    "OC(C)(C)c1onc(c2ncn3c2CN(C)C(c4c3cccc4Cl)=O)n1",
    "CC(=O)Oc1ccccc1C(O)=O",
    "O=C1N=CN=C2NNC=C12",
    "CCCCC[C@H](O)/C=C/[C@H]1[C@H](O)CC(=O)[C@@H]1CCCCCCC(O)=O",
    "CN1C(=O)N(C)c2nc[nH]c2C1=O.CN3C(=O)N(C)c4nc[nH]c4C3=O.NCCN",
    "CCCCc1oc2ccccc2c1C(=O)c3cc(I)c(OCCN(CC)CC)c(I)c3",
    "O.O.O.CC1(C)S[C@@H]2[C@H](NC(=O)[C@H](N)c3ccc(O)cc3)C(=O)N2[C@H]1C(O)=O",
    "CC1(C)S[C@@H]2[C@H](NC(=O)[C@H](N)c3ccccc3)C(=O)N2[C@H]1C(O)=O",
    "C[C@]1(O)CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(C)[C@H]3CC[C@]12C"
]

# Convert molecules to NetworkX graphs
molecule_graphs = [smiles_to_nx_graph(mol) for mol in molecules]

# Calculate the random walk kernel for all pairs of molecules
num_molecules = len(molecule_graphs)
random_walk_kernel_matrix = np.zeros((num_molecules, num_molecules))

for i in range(num_molecules):
    for j in range(num_molecules):
        kernel_value = random_walk_kernel(molecule_graphs[i], molecule_graphs[j], walk_length=2, num_walks=20)
        random_walk_kernel_matrix[i, j] = kernel_value

# print("Random Walk Kernel Matrix:")
# print(random_walk_kernel_matrix)


In [52]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Extracting the 'p_np' column as class labels
class_labels = data['p_np'].tolist()

# Split data into features (graph kernel matrix) and labels
X = random_walk_kernel_matrix
y = class_labels

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Train an SVM classifier
clf = SVC(kernel='sigmoid')
clf.fit(X_train, y_train)

# Predict on test data
y_pred = clf.predict(X_test)

# Evaluate classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.5


### Functions for hashing graphs to strings.


In [54]:
"""
Functions for hashing graphs to strings.
Isomorphic graphs should be assigned identical hashes.
For now, only Weisfeiler-Lehman hashing is implemented.
"""
from hashlib import blake2b

def _hash_label(label, digest_size):
    return blake2b(label.encode("ascii"), digest_size=digest_size).hexdigest()

def _init_node_labels(G, edge_attr, node_attr):
    if node_attr:
        return {u: str(dd[node_attr]) for u, dd in G.nodes(data=True)}
    elif edge_attr:
        return {u: "" for u in G}
    else:
        return {u: str(deg) for u, deg in G.degree()}

def _neighborhood_aggregate(G, node, node_labels, edge_attr=None):
    """
    Compute new labels for given node by aggregating
    the labels of each node's neighbors.
    """
    label_list = []
    for nbr in G.neighbors(node):
        prefix = "" if edge_attr is None else str(G[node][nbr][edge_attr])
        label_list.append(prefix + node_labels[nbr])
    return node_labels[node] + "".join(sorted(label_list))

def weisfeiler_lehman_graph_hash(
    G, edge_attr=None, node_attr=None, iterations=3, digest_size=16
):
    def weisfeiler_lehman_step(G, labels, edge_attr=None):
        """
        Apply neighborhood aggregation to each node
        in the graph.
        Computes a dictionary with labels for each node.
        """
        new_labels = {}
        for node in G.nodes():
            label = _neighborhood_aggregate(G, node, labels, edge_attr=edge_attr)
            new_labels[node] = _hash_label(label, digest_size)
        return new_labels

    # set initial node labels
    node_labels = _init_node_labels(G, edge_attr, node_attr)

    subgraph_hash_counts = []
    for _ in range(iterations):
        node_labels = weisfeiler_lehman_step(G, node_labels, edge_attr=edge_attr)
        counter = Counter(node_labels.values())
        # sort the counter, extend total counts
        subgraph_hash_counts.extend(sorted(counter.items(), key=lambda x: x[0]))

    # hash the final counter
    return _hash_label(str(tuple(subgraph_hash_counts)), digest_size)

In [55]:
from collections import Counter
from rdkit import Chem

# Define a function to convert a SMILES string to a NetworkX graph
def smiles_to_nx_graph(smiles):
    mol = Chem.MolFromSmiles(smiles)
    G = nx.Graph()
    for atom in mol.GetAtoms():
        G.add_node(atom.GetIdx(), atomic_num=atom.GetAtomicNum())
    for bond in mol.GetBonds():
        G.add_edge(bond.GetBeginAtomIdx(), bond.GetEndAtomIdx())
    return G

# Apply Weisfeiler-Lehman hashing to each molecule graph
def weisfeiler_lehman_for_molecules(molecule_graphs, edge_attr=None, node_attr=None, iterations=3, digest_size=16):
    hashes = []
    for graph in molecule_graphs:
        graph_hash = weisfeiler_lehman_graph_hash(graph, edge_attr=edge_attr, node_attr=node_attr, iterations=iterations, digest_size=digest_size)
        hashes.append(graph_hash)
    return hashes

# Convert molecules to NetworkX graphs
molecule_graphs = [smiles_to_nx_graph(mol) for mol in molecules]

# Calculate Weisfeiler-Lehman hash for each molecule graph
wl_hashes = weisfeiler_lehman_for_molecules(molecule_graphs)

print("Weisfeiler-Lehman Hashes:")
for i, hash_val in enumerate(wl_hashes):
    print(f"Molecule {i+1}: {hash_val}")


Weisfeiler-Lehman Hashes:
Molecule 1: fdc62ba403815cbbad163e9a90e4f380
Molecule 2: 59021053eebae767fb2ff217dfd62d2c
Molecule 3: beabbf9aec01fa1dbaa3d6075d144fca
Molecule 4: 0cb1970fabeb6965c279c9fae18bc172
Molecule 5: 2f6f3a1d30ecb2ba14b02bf8cb2feeac
Molecule 6: 2d2cde088a4e720162514658bf8af971
Molecule 7: f1bce8e84ea37cd3f5df8a9e7b1e4019
Molecule 8: 6efd18c0e56b5f45c3688340b24c3502
Molecule 9: c0c9f1824297b949a1c9b9e31720b43c
Molecule 10: d9437e4f1aa9171bfaa70513223d138f
Molecule 11: 7b45a86e36ea37ff77cc1fdb034ccdf6
Molecule 12: b28f416458ad4e13e10fbe112d214c05
Molecule 13: 62b5e9fc8d97351407f8ad44456c2061
Molecule 14: 63a41f239654874f2e05295d36ff4fd3
Molecule 15: 9429dd492cf9667b8b31f9ac23841747
Molecule 16: 4617a153bc030cf8c0f369be6a33da60
Molecule 17: 104894c25635a1f8c2b98b48cf7e76f1
Molecule 18: 548847041ddc78b5a5d5b5693d0d8765
Molecule 19: b5c7c83ad619fb714b9b86891b4234ee
Molecule 20: b795f1fa874b702a7ad46e4bd3020390
Molecule 21: 044a966319483878653be582056ec3e5
Molecule 22: dc01

In [56]:
from sklearn.cluster import KMeans
import numpy as np

# Convert Weisfeiler-Lehman hashes to a numerical representation
X_numerical = np.array([[int(c, 16) for c in hash_val] for hash_val in wl_hashes])

# Perform K-means clustering
n_clusters = 2  #number of clusters
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(X_numerical)

# Print cluster assignments
print("Cluster Assignments:")
for i, label in enumerate(cluster_labels):
    print(f"Molecule {i+1}: Cluster {label}")


Cluster Assignments:
Molecule 1: Cluster 1
Molecule 2: Cluster 0
Molecule 3: Cluster 0
Molecule 4: Cluster 1
Molecule 5: Cluster 1
Molecule 6: Cluster 1
Molecule 7: Cluster 0
Molecule 8: Cluster 0
Molecule 9: Cluster 1
Molecule 10: Cluster 0
Molecule 11: Cluster 0
Molecule 12: Cluster 0
Molecule 13: Cluster 1
Molecule 14: Cluster 0
Molecule 15: Cluster 0
Molecule 16: Cluster 1
Molecule 17: Cluster 1
Molecule 18: Cluster 0
Molecule 19: Cluster 0
Molecule 20: Cluster 1
Molecule 21: Cluster 1
Molecule 22: Cluster 0
Molecule 23: Cluster 1
Molecule 24: Cluster 1
Molecule 25: Cluster 0
Molecule 26: Cluster 1
